In [5]:
import pandas as pd
import yfinance as yf
from datetime import datetime

# Brokers cost dictionary
broker_cost_dict = {
    "010": "0+0.01",
    "011": "100+0.01"
}

# Helper functions
def create_ticker_object(ticker):
    return yf.Ticker(ticker)

def get_asset_ticker_list(file_name="asset_ticker_list.csv"):
    return pd.read_csv(file_name, index_col=False)

def make_asset_list_downloadable(asset_list):
    return asset_list['ticker'].tolist()

def add_history_price_data_to_df(file_name="history.csv"):
    try:
        return pd.read_csv(file_name)
    except FileNotFoundError:
        return pd.DataFrame()

def get_column_order():
    return ['Date', 'Ticker', 'Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits']

def validate_date(date_str):
    """Validate if the date string is in the format 'YYYY-MM-DD HH:MM'."""
    try:
        datetime.strptime(date_str, "%Y-%m-%d %H:%M")
        return True
    except ValueError:
        return False


def validate_price(price_str):
    try:
        price = float(price_str)
        return price > 0
    except ValueError:
        return False

def validate_quantity(quantity_str):
    try:
        quantity = int(quantity_str)
        return quantity > 0
    except ValueError:
        return False
    
def get_broker_code():
    print("Available broker codes:")
    for code in broker_cost_dict.keys():
        print(f"{code}: {broker_cost_dict[code]}")

    while True:
        broker_code = input("Enter broker code: ")
        if broker_code in broker_cost_dict:
            return broker_code
        print("Invalid broker code. Please try again.")

        
def get_history_or_current_price(ticker, period="1d", interval="1d", current=False):
    print(f"Cretaing ticker object for {ticker} ...")
    asset = create_ticker_object(ticker)
    if current:
        current_asset_price = asset.info['currentPrice']
        return current_asset_price
    history = asset.history(period=period, interval=interval)
    return history

def append_trade_to_csv(trade, file_name="trades_register.csv"):
    try:
        # Try to read the existing file
        df_existing = pd.read_csv(file_name)
    except FileNotFoundError:
        # If the file doesn't exist, create a new DataFrame with headers
        df_existing = pd.DataFrame(columns=["Date", "Ticker", "Price", "Quantity", "Broker Code"])

    # Append the new trade
    df_new = pd.DataFrame([trade])
    df_combined = pd.concat([df_existing, df_new], ignore_index=True)

    # Save the combined DataFrame back to the CSV
    df_combined.to_csv(file_name, index=False)
    

def download_initial_history(period="1y", interval="1d", file_name="history.csv"):
    ticker_list = get_asset_ticker_list()
    list_to_be_downloaded = make_asset_list_downloadable(ticker_list)
    all_history = pd.DataFrame()

    for ticker in list_to_be_downloaded:
        print(f"Getting history data for {ticker} ...")
        data = get_history_or_current_price(ticker, period="1y", interval="1d")
        data['Ticker'] = ticker
        all_history = pd.concat([all_history, data])

    # Reset index to make 'Date' a column
    all_history = all_history.reset_index()

    # Reorder columns
    column_order = get_column_order()
    all_history = all_history[column_order]

    # Set 'Date' and 'Ticker' as MultiIndex
    all_history = all_history.set_index(['Date', 'Ticker'])

    # Save history with index
    all_history.to_csv(file_name, index=True)
    
def update_history_with_today_close(file_name="history.csv"):
    asset_list = get_asset_ticker_list()
    ticker_list = make_asset_list_downloadable(asset_list)
    all_history = add_history_price_data_to_df(file_name)

    # If all_history is empty, set it as a DataFrame with the correct columns
    if all_history.empty:
        all_history = pd.DataFrame(columns=get_column_order())
    else:
        # Ensure 'Date' and 'Ticker' are not in the index when loaded from CSV
        all_history = all_history.reset_index()

    column_order = get_column_order()

    for ticker in ticker_list:
        data = get_history_or_current_price(ticker)
        if data.empty:
            print(f"No data for {ticker} today.")
            continue

        # Reset index to make 'Date' a column
        data = data.reset_index()
        data['Ticker'] = ticker

        # Reorder columns
        data = data[column_order]

        # Check if today's data already exists
        if not all_history.empty:
            today = data['Date'].iloc[0]
            mask = (all_history['Ticker'] == ticker) & (all_history['Date'] == today)
            if mask.any():
                print(f"Today's data for {ticker} already exists. Skipping append.")
                continue

        # Append new data
        all_history = pd.concat([all_history, data], ignore_index=True)

        # Remove the oldest record for this ticker (if more than one record exists)
        ticker_mask = all_history['Ticker'] == ticker
        if ticker_mask.sum() > 1:
            oldest_index = all_history[ticker_mask].index.min()
            all_history = all_history.drop(oldest_index)

    # Set 'Date' and 'Ticker' as MultiIndex
    all_history = all_history.set_index(['Date', 'Ticker'])

    # Save history with index
    all_history.to_csv(file_name, index=True)
    
def get_current_prices(ticker_list=None):
    if ticker_list is None:
        ticker_list = get_asset_ticker_list()
        ticker_list = make_asset_list_downloadable(ticker_list)

    current_prices = {}
    for ticker in ticker_list:
        print(f"Getting current price for {ticker} ...")
        current_price = get_history_or_current_price(ticker, current=True)
        current_prices[ticker] = current_price
        print(f"Current price for {ticker} is {current_price}")
        
def add_trade_interactive():
    #initialize_trades_csv()

    print("Enter trade details:")

    # Prompt for date
    while True:
        date_str = input("Date (YYYY-MM-DD HH:MM): ")
        if validate_date(date_str):
            break
        print("Invalid date format. Please use YYYY-MM-DD HH:MM.")

    # Prompt for ticker
    ticker = input("Ticker: ").strip().upper()

    # Prompt for price
    while True:
        price_str = input("Price: ")
        if validate_price(price_str):
            price = float(price_str)
            break
        print("Invalid price. Please enter a positive number.")

    # Prompt for quantity
    while True:
        quantity_str = input("Quantity: ")
        if validate_quantity(quantity_str):
            quantity = int(quantity_str)
            break
        print("Invalid quantity. Please enter a positive integer.")

    # Prompt for broker code
    broker_code = get_broker_code()

    # Create trade dictionary
    trade = {
        "Date": date_str,
        "Ticker": ticker,
        "Price": price,
        "Quantity": quantity,
        "Broker Code": broker_code
    }

    # Append trade to CSV
    append_trade_to_csv(trade)
    print("Trade added successfully!")

    return None

In [3]:
# execute this cell for dowloading initial hostory and create history file
download_initial_history()

Getting history data for AAPL ...
Cretaing ticker object for AAPL ...
Getting history data for INTC ...
Cretaing ticker object for INTC ...
Getting history data for IBM ...
Cretaing ticker object for IBM ...


In [3]:
# execute this cell for download daily prices and update history file
update_history_with_today_close()

Cretaing ticker object for AAPL ...
Cretaing ticker object for INTC ...
Cretaing ticker object for IBM ...


In [ ]:
# execute to add trade
add_trade_interactive()

Enter trade details:


Date (YYYY-MM-DD HH:MM):  2025-12-16 18:12
Ticker:  aapl
Price:  222.99
Quantity:  1000


Available broker codes:
010: 0+0.01
011: 100+0.01


In [ ]:
get_current_prices()